# Report Project 1 - Emezue, Marín, Varela

## Considerations

In order to implement the presented algorithm, we have made the following assumptions:

    (*) Protein orientation in the membrane follows a given pattern when we look at the mean plane generated by the different planes of their aromatic rings, since they are highly hydrophobic.
    (*) Global hydrophobicity is a good measure of whether or not a protein is properly placed in the membrane.
    (*) Membrane width is usually constant across the membrane and has a width of 3 nm [1].
    (*) Normalized hydrophobicies can be used to infer global hydrophobicy [2]. Of course, this can be corrected by proximity or accessibility to the solvent, in this case water, so both ASA and solvation have been implemented as optional considerations.

## Algorithm implementation

Steps:

1) Load structure and select aromatic residues

2) Compute mean director vector from planes of aromatic rings

3) Apply rotation based on the vector

4) Compute optimal Z of membranes 

    (*) if max_it is >1, it either recomputes director vector at each iteration or rotates by a fraction of an angle computed through the number of maximum iterations and the current one. This allows us to change the protein's plane. It chooses as the optimal one the rotated protein and its corresponding Z coordinates that have the lowest global hydrophobicity.
    
    (*) if asa = True, it runs an adaptation of a code from Boscoh to compute Accessible Surface Area for each residue in order to compute hydrophobicity more accurately (improvement with tested proteins has not been significant, though). Code to run this function is present in the .py files of the folder . Source code can be found in: (https://github.com/boscoh/pdbremix/blob/master/pdbremix/asa.py)
    In order to make this work, two new classes had to be defined, adapt the code to these functions and write two new functions. Atomic radii was taken from [3].
    
    (*) if solvation = True, it solvates the protein and chooses only the residues that are in close proximity to water molecules.
    
5) Represent final molecule with embedded membranes
   

In [1]:
from htmd import *
htmd.config(viewer='ngl')
from main_functions import apply_main, dummy_leaflet

Videos from the HTMD2015 workshops are available on the Acellera youtube channel: https://www.youtube.com/user/acelleralive



Using Anaconda Cloud api site https://api.anaconda.org


You are on the latest HTMD version (1.0.16).


In [2]:
###---VARIABLE CONFIGURATION---###
#PDB id of molecule we wish to visualize
molecule_id='2k1l'
#Path to PDB file from OPM
opm_molecule_path=None
#Conditions of aromatic residues: residues and atoms that form ring
conditions = ['resname HIS and (name CE1 or name CD2 or name ND1)',
'(resname TRP or resname PHE or resname TYR) and (name CE2 or name CD2 or name CD1)']
#Number of atoms selected from structure
f_structure_a_n = 3
#Origin and end coordinates of membrane sheet
pmin=[-100,-100]
pmax=[100,100]
#membrane separation
mem_window = 30
#Maximum number of iterations
max_it = None
#ASA calculus switch
asa = False
#Solvation method switch
solvation = False
#Superposition switch
superpose = True
#normalized hydrophobicies according to Monero et al. 1995
my_normalized_hydrophobicies = {'PHE': 1, 'ILE': 0.99, 'TRP': 0.97, 'LEU': 0.97, 'VAL': 0.76, 'MET': 0.74,
                               'TYR': 0.63, 'CYS': 0.49, 'ALA': 0.41, 'THR': 0.13, 'HIS': 0.08, 'GLY': 0,
                                'SER': -0.05, 'GLN': -0.1, 'ARG': -0.14, 'LYS': -0.23, 'ASN': -0.28, 'GLU': -0.31,
                               'PRO': -0.46, 'ASP': -0.55}

In [4]:
###---ALGORITHM IMPLEMENTATION---###

###--STEP 1 & 2: ROTATION OF PROTEIN AND ALIGNMENT OF PROTEIN AND MEMBRANE ON Z AXIS--###
##--INITIALIZATION OF MOLECULE AND FILTERED MOLECULE--##
#Generating molecule object of specified ID
mol=Molecule(molecule_id)
mol.center(sel='all')
#Generating filtering selection
for element in conditions:
    try:
        filtering = filtering + " or (" + element + ")"
    except NameError:
        if len(conditions) > 1:
            element = "(" + element + ")"
        filtering = element
if asa == True:
    import asa_config
    import asa_module
    atoms = asa_module.generate_atom_list(mol)
    areas_dict = asa_module.calculate_asa(atoms, asa_config.distance)
    scale_dict = asa_module.normalize_areas(areas_dict)
elif asa == False:
    scale_dict = None

(mol, z_1, z_2) = apply_main(mol, filtering, f_structure_a_n, my_normalized_hydrophobicies, mem_window, scale_dict, solvation, max_it)

In [5]:
###--STEP 3: CREATION OF MEMBRANES--###
mem1 = dummy_leaflet(pmin,pmax,z_1)
mem2 = dummy_leaflet(pmin,pmax,z_2)
###--STEP 4: VISUALIZATION USING VMD OR NGL--###
allmol=mol.copy()
allmol.append(mem1)
allmol.append(mem2)
allmol.center()
if opm_molecule_path != None:
    opm=Molecule(opm_molecule_path)
    opm.center()
    if superpose != True:
        opm.moveBy([50,0,0])
    allmol.append(opm)
allmol.reps.add(sel='protein', style='NewCartoon')
allmol.reps.add(sel='resname DUM', style='Licorice')
allmol.view()

# TEST RUNS

First of all, we need to import the 'sys' module, since that allows us to look for python scripts in other directories other than the one the code is being executed from. Since all of our test runs are inside a folder named 'test_runs', this first step is required.

* Do note that this syntax only works for *NIX environments.

In [3]:
import sys
sys.path.append('./test_runs')

### Analysis of OPM class 1.1: Alpha-helical polytopic proteins
    
#### Example protein: 4nv2

* Make sure that we have '4nv2.pdb' in the corresponding folder. In the compilation provided, it can be found under the directory './examples'.

The sample code for viewing it is the following:

In [7]:
import test1_1
test1_1.allmol.view()

In this case, the algorithm generates an almost perpendicular protein to the one found in OPM. This is caused by the assumption that the aromatic rings are parallel to the membrane, which is not the case here. Furthermore, the location of the membranes in the protein is wrong as the method to place them is dependent on the orientation of the protein.

#### Example protein: 4nv2 with 20 iterations through angle scanning

In [7]:
import test1_1_it_20_0
test1_1_it_20_0.allmol.view()

As we can see here, even if we try scanning for a protein orientation and membrane position that gives us the overall lowest hydrophobicity, we still get a protein that is perpendicular to the one we are supposed to get.

#### Example protein: 4nv2 with 20 iterations throughdirector vector recomputation

In [8]:
import test1_1_it_20_1
test1_1_it_20_1.allmol.view()

Even if we try to change the rotation to what we defined as optimal, but this time around trying to get solutions with aromatic rings, we do not get better results.

### Analysis of OPM class 1.2: Bitopic proteins
    
#### Example protein: 2k1l

* Make sure that we have '2k1l.pdb' in the corresponding folder. In the compilation provided, it can be found under the directory './examples'.

The sample code for viewing it is the following:

In [26]:
import test1_2
test1_2.allmol.view()

As we can see, for this class of protein, the algorithm appears to position the protein in a plane that is quite close to the one described by OPM.

In this situation, it is proper to see whether or not our use of iterations to look for an angle that rotates the protein in such a way that hydrophobicy is lower works or not, as well as trying the ASA and solvation approaches.

#### Example protein: 2k1l with 20 iterations through angle scanning

In [10]:
import test1_2_it_20_0
test1_2_it_20_0.allmol.view()

As we can see, applying iterations to scan for the angle that minimizes hydrophobicity does not enhance our results.

#### Example protein: 2k1l with 20 iterations through director vector recomputation

In [11]:
import test1_2_it_20_1
test1_2_it_20_1.allmol.view()

Nor does recomputing the mean director vector a set of given times. 

#### Example protein 3m0b

Due to our previous results, further testing in order to see if they can be replicated in other members of the same family is in order.

In [27]:
import test1_2v2
test1_2v2.allmol.view()

As we can see, for another member of the same class of proteins, the protein's plane is no longer close to the one found in OPM.

#### Example protein: 3m0b with 20 iterations through angle scanning

In [15]:
import test1_2v2_it_20_0
test1_2v2_it_20_0.allmol.view()

#### Example protein: 3m0b with 20 iterations through director vector recomputation

In [16]:
import test1_2v2_it_20_1
test1_2v2_it_20_1.allmol.view()

Since the previous finding could not be replicated, it is safe to assume that plane orientation does not work for this class of proteins.

### Analysis of OPM class 1.3: Beta-barrel transmembrane proteins
    
#### Example protein: 2jmm without solvation.

* Make sure that we have '2jmm.pdb' in the corresponding folder. In the compilation provided, it can be found under the directory './examples'.

The sample code for viewing it is the following:

In [8]:
import test1_3
test1_3.allmol.view()

As we can see, the orientation of the protein is almost 45 degrees apart from the one in OPM. Furthermore, the membrane appears narrower than what was anticipated. We will try to use the solvation option to increase the quality of the result.

#### Example protein: 2jmm with solvation.

* Make sure that we have '2jmm.pdb' in the corresponding folder. In the compilation provided, it can be found under the directory './examples'.

The sample code for viewing it is the following:

In [9]:
import test1_3_sol
test1_3_sol.allmol.view()

Solvating: 100% (4/4) [############################################] eta 00:01 /


The use of this option did not produce changes in the result, but still ASA correction will be tried.

#### Example protein: 2jmm with ASA correction

In [30]:
import test1_3_asa
test1_3_asa.allmol.view()

Which, again, does not improve upon the first result.

#### Example protein: 2jmm with 20 iterations through angle scanning

In [31]:
import test1_3_it_20_0
test1_3_it_20_0.allmol.view()

#### Example protein: 2jmm with 20 iterations through director vector recomputation

In [21]:
import test1_3_it_20_1
test1_3_it_20_1.allmol.view()

Surprisingly enough, using the angle scanning iteration does apparently get the protein's plane a bit closer to the one described in OPM. Still, this alone does not seem sufficient to properly place either the membrane (which is fixed) nor put the protein in its correct plane. However, this may be attributed to insufficient angle scanning. To see whether this pattern repeats in other proteins of the same family, further tests are needed.

#### Example protein: 2jmm with 20 iterations through angle scanning with ASA correction

In [32]:
import test1_3_it_20_0_asa
test1_3_it_20_0_asa.allmol.view()

As we can see here, computing ASA and joining it with the optimal hydrophobicy with angle scanning does not apparently yield much better results than simply angle scanning. However, without numerical comparison, this is not possible to truly know.

#### Example protein: 2x27

To further explore the observations made with the other protein of its family, 2x27 has been chosen.

In [33]:
import test1_3v2
test1_3v2.allmol.view()

As we can see, the initial approach of placing it in a plane with director vector as the mean of the planes formed with its aromatic rings does again give us a plane that appears to be close to 45 degrees off.

#### Example protein: 2x27 with 20 iterations through angle scanning

In [34]:
import test1_3v2_it_20_0
test1_3v2_it_20_0.allmol.view()

However, increasing the iterations to scan for the angle which returns the lowest global hydrophobicity does not yield better results in this case. Arguably, they might be considered worse.

#### Example protein: 2x27 with 20 iterations throughdirector vector recomputing

In [35]:
import test1_3v2_it_20_1
test1_3v2_it_20_1.allmol.view()

And the same can be said about scanning performed through director vector recalculus.


### Analysis of OPM class 2.3: Alpha/Beta monotopic/peripheral superfamilies 
#### Example protein: 1auo

* Make sure that we have '1aou.pdb' in the corresponding folder. In the compilation provided, it can be found under the directory './examples'.

The sample code for viewing it is the following:

In [5]:
import test2_3
test2_3.allmol.view()

As can be seen in the image, this protein should be in contact with only one of the sheets from the membrane. However, our approximation places inserted in the middle of the membrane. The computation of the molecule's hydrophobicity is not able to solve this type of proteins which barely contact the membrane.

#### Example protein: 1auo with 10 iterations through angle scanning

The sample code for viewing it is the following:

In [4]:
import test2_3_it_10_0
test2_3_it_10_0.allmol.view()

Increasing the number of iterations does not seem to give better results. Due to the protein's positioning, both ASA and solvation procedures will be tested.

#### Example protein: 1auo with 10 iterations through angle scanning with solvation correction

In [5]:
import test2_3_it_10_0_sol
test2_3_it_10_0_sol.allmol.view()

As we can see, solvation does not improve the results either, so ASA will be tried.

#### Example protein: 1auo with 10 iterations through angle scanning with ASA correction

In [6]:
import test2_3_it_10_0_asa
test2_3_it_10_0_asa.allmol.view()

As we can see, none of the corrections applied allows us to properly predict protein topicity.

### Analysis of OPM class 3.1:  Alpha-helical peptides
#### Example protein: 2rnd

Make sure that we have '2rnd.pdb' in the corresponding folder. In the compilation provided, it can be found under the directory './examples'.

The sample code for viewing it is the following:


In [7]:
import test3_1
test3_1.allmol.view()

Like in the previous case, the program fails to situate the protein in contact with only one membrane. On top of that, in this case it can be clearly seen that the rotation angle that has been applied is not correct

### Conclusions

At the light of our results, we can say the following:

- Aromatic rings are not a self-sufficient or good enough criteria to determine membrane positioning.

- Neither of the methods we came up with in order to improve membrane placement produced highly better results.

    - In the case of iterative search, we considered global hydrophobicity as a good indicator of how good an alignment between the protein and the membrane was. However, what we did not consider was that maybe some of the optimal conformations were not actually possible due to spatial constraints. We can see that some of our results actually place the entirety of the protein inside the membrane, which may not be possible for large proteins. At present, it is unknown whether filtering these solutions out of the solution space would improve the algorithm. However, the number of observations is low and not representative of the entirety of protein variation that can be found, so it is just a baseless conjecture.
    - In the case of ASA, we thought it might help when dealing with hydrophobic cores surrounded by hydrophilic residues, but the improvement is not clearly visible and we lack statistical power to test if it is significant.
    - In the case of solvation, we can say the same as ASA.

It could be that the method we used to compute hydrophobicity was not a good approach and the improvement methods that we applied could not correct the fact that we started with a wrong assumption. Another explanation could be that the normalized hydrophobicity coefficients that we used where not adequate for this kind of computation. These have been taken from the bibliography, yet instead of taking them, one could simply obtain them through computing the RMSD between a large enough set of proteins taken from OPM and the ones that we have rotated (with the aromatic ring criteria or any other to position them) and then optimizing by this RMSD and deriving the set of coefficients that would make this result possible. Of course, these would need to be averaged either by the whole set of proteins or by subsets, whichever works best.

The possibility of this last solution giving artefactual results also needs to be taken into consideration, but considering that similar proteins also have similar biochemical propiertries is not too far of a stretch from other approximations already done.

Something interesting to see is whether or not our algorithm is at least capable of properly predicting protein topicity, but seeing the results yielded so far, this is unlikely.

In conclusion, we can say that the approach we have tried in order to compute the rotation angle that needs to be applied to the protein has not proved successful. Regarding the Z positioning of the membrane, we achieve passable results for some kinds of integral proteins, while the positioning for peripheral proteins is systematically bad.

### Bibliography

[1] http://book.bionumbers.org/what-is-the-thickness-of-the-cell-membrane/
    
[2] Monera, O. D., Sereda, T. J., Zhou, N. E., Kay, C. M., & Hodges, R. S. Relationship of sidechain
hydrophobicity and alpha-helical propensity on the stability of the single-stranded amphipathic 
alpha-helix. Journal of Peptide Science : An Official Publication of the European Peptide
Society, 1(5), 319–29. http://doi.org/10.1002/psc.310010507

[3] Daniel Seeliger and Bert L. de Groot Atomic contacts in protein structures. A detailed analysis of atomic radii, packing and overlaps